In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
all_tox = os.listdir('/content/drive/My Drive/ClinicalTrials/uniTox Ratings/FinaluniTox')

In [ ]:
tox_names = [this_tox.split("_")[1] for this_tox in all_tox[0:8]]
print(tox_names)

In [ ]:
first_tox = all_tox[0]
tox_df = pd.read_csv(os.path.join('/content/drive/My Drive/ClinicalTrials/uniTox Ratings/FinaluniTox',first_tox))
tox_df = tox_df[["generic_name","less_determination","binary_determination"]]
tox_df.columns = ["generic_name","less_determination_"+tox_names[0],"binary_determination_"+tox_names[0]]
desired_order_less = ["No","Less","Most"]
desired_order_binary = ["No","Yes"]
less_counts_df = tox_df["less_determination_"+tox_names[0]].value_counts().reindex(desired_order_less).rename(tox_names[0])
binary_counts_df = tox_df["binary_determination_"+tox_names[0]].value_counts().reindex(desired_order_binary).rename(tox_names[0])
tox_df["binary_determination_int_"+tox_names[0]] = tox_df["binary_determination_"+tox_names[0]].apply(lambda x: 1 if x == "Yes" else 0)

for tox, name in zip(all_tox[1:],tox_names[1:]):
  this_tox_df = pd.read_csv(os.path.join('/content/drive/My Drive/ClinicalTrials/uniTox Ratings/FinaluniTox',tox))
  this_tox_df = this_tox_df[["generic_name","less_determination","binary_determination"]]
  this_tox_df.columns = ["generic_name","less_determination_"+name,"binary_determination_"+name]
  print(this_tox_df["less_determination_"+name].value_counts().reindex(desired_order_less))
  print(this_tox_df["binary_determination_"+name].value_counts().reindex(desired_order_binary))
  less_counts_df = pd.concat([less_counts_df,this_tox_df["less_determination_"+name].value_counts().reindex(desired_order_less).rename(name)],axis=1)
  binary_counts_df = pd.concat([binary_counts_df,this_tox_df["binary_determination_"+name].value_counts().reindex(desired_order_binary).rename(name)],axis=1)
  this_tox_df["binary_determination_int_"+name] = this_tox_df["binary_determination_"+name].apply(lambda x: 1 if x == "Yes" else 0)
  tox_df = tox_df.merge(this_tox_df,on="generic_name")


In [ ]:
alpha_tox_df = tox_df[["binary_determination_"+this_tox_name for this_tox_name in tox_names]]

In [ ]:
alpha_tox_df.columns = alpha_tox_df.columns.str.replace("binary_determination_","")

In [ ]:
alpha_tox_df = alpha_tox_df.reindex(sorted(alpha_tox_df.columns), axis=1)


In [ ]:
alpha_tox_df

In [ ]:
alpha_tox_df = alpha_tox_df.apply(lambda x: (x == 'Yes').astype(int))



In [ ]:
alpha_tox_df

In [ ]:
import numpy as np
from scipy import stats
corrs = np.zeros((len(alpha_tox_df.columns),len(alpha_tox_df.columns)))
for i in range(0,len(alpha_tox_df.columns)):
  for j in range(0,len(alpha_tox_df.columns)):
    if i != j:
      corrs[i,j] = stats.pearsonr(alpha_tox_df.iloc[:,i],alpha_tox_df.iloc[:,j])[0]
    else:
      corrs[i,j] = np.nan

In [ ]:
print(alpha_tox_df.columns,"    ")
print(np.array2string(corrs, formatter={'float_kind': lambda x: f'{x:7.2f}'}))